In [1]:
from astropy.io import fits
import clevar
from clevar.match_metrics.recovery import ClCatalogFuncs as r_cf
from clevar.match_metrics.distances import ClCatalogFuncs as d_cf
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf
import clevar.match_metrics.scaling as scaling

import pandas as pd
from astropy.table import Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

from getdist import plots
from getdist.mcsamples import  MCSamples
import getdist
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))

#import pygtc
import pandas as pd

GetDist Version: 1.4.7, Matplotlib version: 3.8.2


In [2]:
hdul_halos = fits.open("../halos/halos.fits")
hdul_ame = fits.open("richness_mcut_SNR 12_ame.fits")

logMmin = 13
logMmax = 15

table_halos = hdul_halos[1].data
table_ame = hdul_ame[1].data
table_halos = table_halos[(table_halos['upid'] == -1)  &  (table_halos['Mvir'] >= 10**logMmin) & (table_halos['Mvir'] <= 10**logMmax)]
table_halos['Mvir'] = table_halos['Mvir']/0.6774

print(table_halos.size)
print(table_ame.size)

print(table_ame.columns)

689973
23751
ColDefs(
    name = 'ID'; format = 'I'
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'z'; format = 'D'
    name = 'z_err'; format = 'D'
    name = 'SNR'; format = 'D'
    name = 'SNR_ORIG'; format = 'D'
    name = 'richness'; format = 'D'
    name = 'radius'; format = 'D'
    name = 'rank'; format = 'J'
    name = 'Rc'; format = 'D'
    name = 'R_ame'; format = 'D'
    name = 'R_ame_err'; format = 'D'
)


In [3]:
catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R'], mass=table_halos['Mvir'])

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'], mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err'] ,z_err=table_ame['z_err'] , )
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_ame, out_dir='temp')

halos_match, detections_match = clevar.match.get_matched_pairs(catalog_halos, catalog_ame, 'cross')

detections_match

/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/catalog/tagdata.py:278: UserWarning: id column missing, additional one is being created.
  warnings.warn(f"{id_name} column missing, additional one is being created.")


Halo catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    689,973
 * multiple (self):  33,356
 * multiple (other): 33,356
 * unique (self):    23,639
 * unique (other):   23,639
 * cross:            22,282

Detection catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    23,751
 * multiple (self):  23,639
 * multiple (other): 23,639
 * unique (self):    23,639
 * unique (other):   23,639
 * cross:            22,282


id,ra,dec,z,mass,mass_err,z_err,mt_self,mt_other,mt_multi_self,mt_multi_other,mt_cross
str5,float64,float64,float64,float64,float64,float64,object,object,object,object,object
3464,224.2628875340314,61.028825506020304,0.0683782529345174,37.0,2.2375820282915515,0.0083782529345174,199,199,['199'],['199'],199
19744,227.04884922802296,69.05066906371957,0.0696005956063757,18.0,2.295499922311768,0.0096005956063757,202,202,['202'],['202'],202
9013,226.6554504120344,60.847267680598335,0.0669793100502585,22.0,2.980191779063892,0.0069793100502585,203,203,['203'],['203'],203
14298,226.36106198202708,59.79601040562029,0.0639392094776381,26.0,4.520390850357965,0.0039392094776381,204,204,"['200', '204']","['200', '204']",204
6956,196.56822311398383,56.78175408392197,0.0517186429521282,41.0,1.855297395747288,0.0017186429521282,209,209,['209'],['209'],209
20249,203.6522381950678,44.957848865272354,0.0481005617843885,66.0,4.070279453141599,0.0018994382156114,237,237,['237'],['237'],237
21218,202.3165898936235,48.85604116665247,0.0488028499804194,49.0,3.2170573054682543,0.0011971500195805,238,238,['238'],['238'],238
17390,202.47958752311607,54.66305059606346,0.046599787784135,35.0,2.9658536713735564,0.0034002122158649,239,239,['239'],['239'],239
15499,197.81432614479684,55.78723163875684,0.0503115080850006,27.0,3.858080406972704,0.0003115080850006,240,240,['240'],['240'],240


In [4]:
zbins = np.linspace(0, 0.664, 20)
mbins = np.logspace(13.0, 15, 4)
#mbins = np.logspace(0.0, 3, 4)

print(halos_match.size)
print(detections_match.size)

#mbins = np.append(np.logspace(13.5, 15, 4), 1e15)
#info = s_cf.plot_density(halos_match, detections_match, 'cross', col='z', add_err=True, add_fit=True, fit_bins1=20, fit_log=True)
#plt.show()


info = s_cf.plot(halos_match, detections_match, 'cross', col='mass', xscale='log', yscale='log')
plt.savefig("mass_rich.jpg")
plt.show()

info = s_cf.plot(detections_match, halos_match, 'cross', col='mass', xscale='log', yscale='log')
plt.savefig("rich_mass.jpg")
plt.show()

22282
22282


/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/match_metrics/scaling/aux_funcs.py:143: UserWarning: Too few data points for fit!
  warnings.warn("Too few data points for fit!")
/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/match_metrics/scaling/aux_funcs.py:143: UserWarning: Too few data points for fit!
  warnings.warn("Too few data points for fit!")


In [5]:
clusters_dt = Table([halos_match['mass'], detections_match['mass'], halos_match['z']], names=('Mvir', 'richness' , 'z'))

lnM = np.log(clusters_dt["Mvir"])  # ln(mass)
z = clusters_dt["z"]  # redshift
lnR = np.log(clusters_dt["richness"])  # ln(richness)

#bins:
d = 0.05
z_bins = int((max(z) - min (z)) // d  + 1)

d_m = 0.25
m_bins = int((max(lnM) - min (lnM)) // d_m + 1)

print(f'Number of z bins: {z_bins}.\nNumber of lnM bins: {m_bins}.\nTotal bins: {z_bins * m_bins}.')

Number of z bins: 13.
Number of lnM bins: 19.
Total bins: 247.


In [6]:
# redshift bins
z_0 = 0.0
z_1 = 0.05
halos_bin_z =[]

for i in range(13):
    cut_z = np.logical_and (clusters_dt['z'] > z_0, clusters_dt['z'] < z_1)
    halos_bin_z.append(clusters_dt[cut_z])
    z_0 = z_0 + 0.05
    z_1 = z_1 + 0.05

# mass bins
label = []
halos_bin_mz =[]
for i in range(13):
    
    lnM_0 = min(lnM)
    lnM_1 = min(lnM) + 0.25
    for j in range(31):
        
        cut = np.logical_and (np.log(halos_bin_z[i]["Mvir"]) > lnM_0, np.log(halos_bin_z[i]["Mvir"]) < lnM_1)
        halos_bin_mz.append(halos_bin_z[i][cut])
        #label.append(f"{min(halos_bin_z[i]['redshift_true']):.3f} < z < {max(halos_bin_z[i]['redshift_true']):.3f}\n{lnM_0:.3f} < lnM < {lnM_1:.3f}")
        
        lnM_0 = lnM_0 + 0.25
        lnM_1 = lnM_1 + 0.25

print(f'Total bins in halos_bin_mz: {len(halos_bin_mz)}')

Total bins in halos_bin_mz: 403


In [7]:
lnM_binned, z_binned, lnR_binned = [], [], [] 

for i in range(len(halos_bin_mz)):
    
    halos = halos_bin_mz[i]
    lnM_binned.append(np.log(halos["Mvir"]))
    z_binned.append(halos["z"])
    lnR_binned.append(np.log(halos["richness"]))   

lnR_mean, lnM_mean, z_mean = [np.mean(l) for l in lnR_binned if len(l) > 1], [np.mean(l) for l in lnM_binned if len(l) > 1], [np.mean(k) for k in z_binned if len(k) > 1]
lnR_std, lnM_std, z_std = [np.std(l) for l in lnR_binned if len(l) > 1], [np.std(l) for l in lnM_binned if len(l) > 1], [np.std(k) for k in z_binned if len(k) > 1]


halos_mean = Table([np.exp(np.array(lnR_mean)), np.exp(np.array(lnM_mean)), z_mean],
           names=('richness', 'Mvir', 'z'))

halos_std = Table([np.exp(np.array(lnR_std)), np.exp(np.array(lnM_std)), z_std],
           names=('richness', 'Mvir', 'z'))

print(f'Total bins in halos_mean: {len(halos_mean)}')

Total bins in halos_mean: 219


In [8]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_mean  # MÉDIA DE LN(R)
xs = z_mean # MÉDIA DE Z
ys = lnM_mean # MÉDIA DE LN(M)

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='rainbow')
ax1.set_xlabel('z', fontsize = 14)
ax1.set_ylabel('lnM', fontsize = 14)
fig.colorbar(p3, ax=ax1).set_label(label=r'$ \mu_{\ln \lambda} $',size=14)

p4 = ax2.scatter(ys ,zs,  c= xs, cmap='rainbow')
ax2.set_xlabel('lnM', fontsize = 14)
ax2.set_ylabel(r'$ \mu_{\ln \lambda} $', fontsize = 14)

fig.colorbar(p4, ax=ax2).set_label(label='z',size=14)


p5 = ax3.scatter(xs, zs , c=ys, cmap='rainbow') 
ax3.set_xlabel('z', fontsize = 14)
ax3.set_ylabel(r'$ \mu_{\ln \lambda} $',  fontsize = 14)
fig.colorbar(p5, ax=ax3).set_label(label='lnM',size=14)


fig.suptitle(r'$ \mu_{\ln \lambda} $ em função de lnM e z ', size = 18, y=1.10)

plt.savefig("mass_richness_mean_data.jpg")
plt.show()

In [9]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_std  # MÉDIA DE LN(R)
xs = z_mean # MÉDIA DE Z
ys = lnM_mean # MÉDIA DE LN(M)

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='rainbow')
ax1.set_xlabel('z', fontsize = 14)
ax1.set_ylabel('lnM', fontsize = 14)
fig.colorbar(p3, ax=ax1).set_label(label=r'$ \sigma{\ln \lambda} $',size=14)

p4 = ax2.scatter(ys ,zs,  c= xs, cmap='rainbow')
ax2.set_xlabel('lnM', fontsize = 14)
ax2.set_ylabel(r'$ \sigma{\ln \lambda} $', fontsize = 14)

fig.colorbar(p4, ax=ax2).set_label(label='z',size=14)


p5 = ax3.scatter(xs, zs , c=ys, cmap='rainbow') 
ax3.set_xlabel('z', fontsize = 14)
ax3.set_ylabel(r'$ \sigma{\ln \lambda} $',  fontsize = 14)
fig.colorbar(p5, ax=ax3).set_label(label='lnM',size=14)


fig.suptitle(r'$ \sigma{\ln \lambda} $ em função de lnM e z ', size = 18, y=1.10)

plt.savefig("mass_richness_std_data.jpg")
plt.show()

## Linear Model

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
    \mu(M,z) = \mu_0 + \mu_M\ln\frac{M}{M_0} + \mu_z\ln\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + \sigma_M\ln\frac{M}{M_0} + \sigma_z\ln\frac{1+z}{1+z_0} 
$$


In [10]:
cluster_m = Nc.ClusterMassAscaso(lnRichness_min = 4 ,lnRichness_max = 5.5)
cluster_m.param_set_by_name("mup0",1.8649)
cluster_m.param_set_by_name("mup1", 0.4)
cluster_m.param_set_by_name("mup2", 0)
cluster_m.param_set_by_name("sigmap0", 1.06)
cluster_m.param_set_by_name("sigmap1", 0.07)
cluster_m.param_set_by_name("sigmap2", 0.0)
cluster_m.param_set_by_name("cut", 10000)

lnM_v = Ncm.Vector.new(len(clusters_dt))
z_v = Ncm.Vector.new(len(clusters_dt))
rich_v = Ncm.Vector.new(len(clusters_dt))

for i, mass in enumerate(clusters_dt["Mvir"]):
    lnM_v.set(i, np.log(mass))

for i, z in enumerate(clusters_dt["z"]):
    z_v.set(i, z)

for i, rich in enumerate(clusters_dt["richness"]):
    rich_v.set(i, np.log(rich))

dmr = Nc.DataClusterMassRich.new()
dmr.set_data(lnM_v, z_v, rich_v)

mset = Ncm.MSet()
mset.set(cluster_m)
dmr.m2lnL_val(mset)  
mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

fixed_parameters = ['cut']

for par in fixed_parameters:
    mset["NcClusterMass"].param_set_desc(par, {"fit": False})

#Data
dset = Ncm.Dataset.new()
dset.append_data(dmr)

#Likelihood
lh = Ncm.Likelihood.new(dset)



#Fit
fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
fit.log_info()
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
fit.log_info()

nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("linear.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  1.8649              [FREE]
#   -    mup1[01]:  0.4                 [FREE]
#   -    mup2[02]:  0                   [FREE]
#   - sigmap0[03]:  1.06                [FREE]
#   - sigmap1[04]:  0.07                [FREE]
#   - sigmap2[05]:  0                   [FREE]
#   -     cut[06]:  10000               [FIXED]
#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#........................................................

In [11]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

#lnR model, z_mean, y_mean
xs = z_mean
ys = lnM_mean
zs = np.array([cluster_m.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

p1 =ax1.scatter(xs, ys, zs, c=zs, cmap='cool')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
fig.colorbar(p1, ax=ax1, label='<LnR| M, z>')

ax2.scatter(lnM_mean, lnR_mean, c='k', label='lnR médio')
p2 = ax2.scatter(ys, zs , c= xs, cmap='cool')
ax2.set_xlabel('lnM')
ax2.set_ylabel('<LnR| M, z>')
fig.colorbar(p2, ax=ax2, label='z')
ax2.legend()

ax3.scatter(z_mean, lnR_mean, c='k', label='lnR médio')
p3 = ax3.scatter(xs, zs , c=ys, cmap='cool') 
ax3.set_xlabel('z')
ax3.set_ylabel('<LnR| M, z>')
fig.colorbar(p3, ax=ax3, label='lnM')
ax3.legend()
fig.suptitle('Linear Model Mean')
plt.savefig("linear_model_mean.jpg")
plt.show()

In [12]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

#lnR model, z_mean, y_mean
xs = z_mean
ys = lnM_mean
zs = np.array( [cluster_m.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

p1 =ax1.scatter(xs, ys, zs, c=zs, cmap='cool')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
fig.colorbar(p1, ax=ax1, label=r'$\sigma(M,z)$')

ax2.scatter(lnM_mean, lnR_std, c='k', label='lnR std')
p2 = ax2.scatter(ys, zs , c= xs, cmap='cool')
ax2.set_xlabel('lnM')
ax2.set_ylabel(r'$\sigma(M,z)$')
fig.colorbar(p2, ax=ax2, label='z')
ax2.legend()

ax3.scatter(z_mean, lnR_std, c='k', label='lnR std')
p3 = ax3.scatter(xs, zs , c=ys, cmap='cool') 
ax3.set_xlabel('z')
ax3.set_ylabel(r'$\sigma(M,z)$')
fig.colorbar(p3, ax=ax3, label='lnM')
ax3.legend()
fig.suptitle('Linear Model Standard Deviation')
plt.savefig("linear_model_std.jpg")
plt.show()

In [13]:
data_fit_full = pd.DataFrame(fits.open("linear.fits")[1].data).iloc[:, 1:13].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
]
labels=[r"\mu_0", r"\mu_{M1}" ,r"\mu_{Z1}",
        r"\sigma_0", r"\sigma_{M1}", r"\sigma_{Z1}"]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples3.removeBurn(0.3)

# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples3],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
        legend_labels=["Linear", ],
)
plt.savefig('linear_corner_plot.png')
plt.show()

Removed no burn in


## Quadratic Model

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\begin{split}
   \mu(M,z) = \mu_0 + 
    \mu_{M1} \ln\left( \frac{M}{M_0} \right) +
    \mu_{M2} (\ln\left(\frac{M}{M_0} \right))^2 +\\
    + \mu_{z1} \ln\left(\frac{1 + z}{1 + z_0}\right) +
    \mu_{z2} (\ln\left(\frac{1 + z}{1 + z_0}\right))^2 +\\
    + \mu_{Mz} \ln\left(\frac{1 + z}{1 + z_0}\right)\ln\left(\frac{M}{M_0} \right);
\end{split}
$$
variance is
$$
\begin{split}
 \sigma(M,z) = \sigma_0 + 
    \sigma_{M1} \ln\left( \frac{M}{M_0} \right) +
    \sigma_{M2} (\ln\left(\frac{M}{M_0} \right))^2 +\\
    + \sigma_{z1} \ln\left(\frac{1 + z}{1 + z_0}\right) +
    \sigma_{z2} (\ln\left(\frac{1 + z}{1 + z_0}\right))^2 +\\
    + \sigma_{Mz} \ln\left(\frac{1 + z}{1 + z_0}\right)\ln\left(\frac{M}{M_0} \right);
\end{split}
$$


In [14]:
cluster_m = Nc.ClusterMassLnrichExt(lnRichness_min = 0 ,lnRichness_max = 5.5)
cluster_m.param_set_by_name("cut", 10000)

lnM_v = Ncm.Vector.new(len(clusters_dt))
z_v = Ncm.Vector.new(len(clusters_dt))
rich_v = Ncm.Vector.new(len(clusters_dt))

for i, mass in enumerate(clusters_dt["Mvir"]):
    lnM_v.set(i, np.log(mass))

for i, z in enumerate(clusters_dt["z"]):
    z_v.set(i, z)

for i, rich in enumerate(clusters_dt["richness"]):
    rich_v.set(i, np.log(rich))

dmr = Nc.DataClusterMassRich.new()
dmr.set_data(lnM_v, z_v, rich_v)

mset = Ncm.MSet()
mset.set(cluster_m)
dmr.m2lnL_val(mset)  
mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

fixed_parameters = ['A0','cut', 'cutM1', 'cutZ1']

for par in fixed_parameters:
    mset["NcClusterMass"].param_set_desc(par, {"fit": False})
    
#Data
dset = Ncm.Dataset.new()
dset.append_data(dmr)

#Likelihood
lh = Ncm.Likelihood.new(dset)

experiment = Ncm.ObjDictStr()
experiment.set("likelihood", lh)
experiment.set("model-set", mset)
ser = Ncm.Serialize.new(Ncm.SerializeOpt.CLEAN_DUP)
ser.dict_str_to_yaml_file(experiment, "quadratic.yaml")

#Fit
fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
fit.log_info()
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
fit.log_info()

nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("quadratic.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.19                [FREE]
#   -    muM1[01]:  0.868588963806504   [FREE]
#   -    muZ1[02]: -0.304006137332276   [FREE]
#   -    muM2[03]:  0                   [FREE]
#   -    muZ2[04]:  0                   [FREE]
#   -    muMZ[05]:  0                   [FREE]
#   -  sigma0[06]:  0.33                [FREE]
#   - sigmaM1[07]: -0.0347435585522601  [FREE]
#   - sigmaZ1[08]:  0                   [FREE]
#   - sigmaM2[09]:  0                   [FREE]
#   - sigmaZ2[10]:  0                   [FREE]
#   - sigmaMZ[11]:  0                   [FREE]
#   -      A0[12]:  0                   [FIX

In [15]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

#lnR model, z_mean, y_mean
xs = z_mean
ys = lnM_mean
zs = np.array([cluster_m.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

p1 =ax1.scatter(xs, ys, zs, c=zs, cmap='cool')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
fig.colorbar(p1, ax=ax1, label='<LnR| M, z>')

ax2.scatter(lnM_mean, lnR_mean, c='k', label='lnR médio')
p2 = ax2.scatter(ys, zs , c= xs, cmap='cool')
ax2.set_xlabel('lnM')
ax2.set_ylabel('<LnR| M, z>')
fig.colorbar(p2, ax=ax2, label='z')
ax2.legend()

ax3.scatter(z_mean, lnR_mean, c='k', label='lnR médio')
p3 = ax3.scatter(xs, zs , c=ys, cmap='cool') 
ax3.set_xlabel('z')
ax3.set_ylabel('<LnR| M, z>')
fig.colorbar(p3, ax=ax3, label='lnM')
ax3.legend()
fig.suptitle('Quadratic Model')
plt.savefig("quadratic_model_mean.jpg")
plt.show()

In [16]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

#lnR model, z_mean, y_mean
xs = z_mean
ys = lnM_mean
zs = np.array( [cluster_m.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

p1 =ax1.scatter(xs, ys, zs, c=zs, cmap='cool')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
fig.colorbar(p1, ax=ax1, label=r'$\sigma(M,z)$')

ax2.scatter(lnM_mean, lnR_std, c='k', label='lnR std')
p2 = ax2.scatter(ys, zs , c= xs, cmap='cool')
ax2.set_xlabel('lnM')
ax2.set_ylabel(r'$\sigma(M,z)$')
fig.colorbar(p2, ax=ax2, label='z')
ax2.legend()

ax3.scatter(z_mean, lnR_std, c='k', label='lnR std')
p3 = ax3.scatter(xs, zs , c=ys, cmap='cool') 
ax3.set_xlabel('z')
ax3.set_ylabel(r'$\sigma(M,z)$')
fig.colorbar(p3, ax=ax3, label='lnM')
ax3.legend()
fig.suptitle('Quadratic Model Standard Deviation')
plt.savefig("quadratic_model_std.jpg")
plt.show()

In [17]:
data_fit_full = pd.DataFrame(fits.open("quadratic.fits")[1].data).iloc[:, 1:13].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
]
labels=[r"\mu_0", r"\mu_{M1}", r"\mu_{M2}", r"\mu_{Z1}",r"\mu_{Z2}", r"\mu_{MZ}",
        r"\sigma_0", r"\sigma_{M1}", r"\sigma_{M2}", r"\sigma_{Z1}", r"\sigma_{Z2}", r"\sigma_{MZ}"]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples3.removeBurn(0.3)

# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples3],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
        legend_labels=["Quadratic", ],
)
plt.savefig('quadratic_corner_plot.png')
plt.show()

Removed no burn in
